In [1]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
import pandas as pd

In [2]:
class PhoBERTMultiTaskModel(nn.Module):
    def __init__(self, model_name, num_aspects):
        super(PhoBERTMultiTaskModel, self).__init__()

        self.bert = AutoModel.from_pretrained(model_name, output_hidden_states=True)
        self.hidden_size = self.bert.config.hidden_size * 4  # Concatenating last 4 layers
        self.dropout = nn.Dropout(0.2)

        # ✅ Binary aspect detection (present or not)
        self.aspect_detectors = nn.ModuleList([
            nn.Linear(self.hidden_size, 2) for _ in range(num_aspects)
        ])

        # ✅ Sentiment classification (4 classes)
        self.aspect_classifiers = nn.ModuleList([
            nn.Linear(self.hidden_size, 4) for _ in range(num_aspects)
        ])

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.hidden_states
        pooled_output = torch.cat([hidden_states[-i][:, 0, :] for i in range(1, 5)], dim=-1)
        pooled_output = self.dropout(pooled_output)

        # ✅ Aspect detection (Binary)
        aspect_presence = [detector(pooled_output) for detector in self.aspect_detectors]
        aspect_presence = torch.stack(aspect_presence, dim=1)  # (batch_size, num_aspects, 2)

        # ✅ Sentiment classification (4 classes)
        aspect_sentiment = [classifier(pooled_output) for classifier in self.aspect_classifiers]
        aspect_sentiment = torch.stack(aspect_sentiment, dim=1)  # (batch_size, num_aspects, 4)

        return aspect_presence, aspect_sentiment

In [3]:
import torch
from transformers import AutoTokenizer, AutoModel

# Define model path
model_path = "/kaggle/input/phobert_multitask-v2/transformers/default/1/phobert_multitask-V2"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load fine-tuned model
model = PhoBERTMultiTaskModel(model_name="vinai/phobert-base", num_aspects=34)

# ✅ Force loading on CPU to avoid NCCL issues
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load(f"{model_path}/pytorch_model.bin", map_location=device))
model.to(device)
model.eval()  # Set to evaluation mode

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

<ipython-input-3-562b68f01bc8>:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"{model_path}/pytorch_model.bin", map_location=device))


PhoBERTMultiTaskModel(
  (bert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNor

In [9]:
label_keys = [
    "FACILITIES#CLEANLINESS", "FACILITIES#DESIGN&FEATURES", "FACILITIES#MISCELLANEOUS", "FACILITIES#QUALITY",
    "FOOD&DRINKS#PRICES", "FOOD&DRINKS#STYLE&OPTIONS", "HOTEL#COMFORT", "HOTEL#GENERAL", "HOTEL#PRICES",
    "LOCATION#GENERAL", "ROOMS#COMFORT", "ROOMS#MISCELLANEOUS", "ROOMS#PRICES", "ROOM_AMENITIES#CLEANLINESS",
    "ROOM_AMENITIES#GENERAL", "ROOM_AMENITIES#MISCELLANEOUS", "SERVICE#GENERAL"
]  # Ensure all aspects from training are listed

def predict(text):
    """Tokenizes input text, runs inference, and returns predicted aspect presence & sentiment."""

    # ✅ Tokenize input text and move to the same device as the model
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=256)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # ✅ Move to correct device

    if "token_type_ids" in inputs:
        del inputs["token_type_ids"]

    # Run inference
    with torch.no_grad():
        aspect_preds, sentiment_preds = model(**inputs)  # ✅ Model outputs two predictions

    # ✅ Convert logits to class predictions
    aspect_probs = torch.sigmoid(aspect_preds)  # Convert logits to probabilities (binary classification)
    aspect_predictions = (aspect_probs > 0.5).cpu().numpy().flatten().tolist()  # Convert to 0 or 1

    sentiment_labels = torch.argmax(sentiment_preds, dim=-1).cpu().numpy().flatten().tolist()  # Convert logits to class indices

    # ✅ Sentiment mapping
    sentiment_map = {0: "None", 1: "Positive", 2: "Negative", 3: "Neutral"}

    # ✅ Prepare output
    predictions = {}
    for aspect, aspect_present, sentiment in zip(label_keys, aspect_predictions, sentiment_labels):
        if aspect_present == 1:  # ✅ Only include aspects that are detected
            predictions[aspect] = sentiment_map[int(sentiment)]

    return predictions  # ✅ Returns dictionary of detected aspects & their sentiment


In [13]:
text = input()
res = predict(text)
res

 khách_sạn khá sạch_sẽ nhân_viên nhiệt_tình thân_thiện gần biển phạm văn đồng có_thể đi bộ ra biển được wifi tốt giá giặt là khá rẻ điểm trừ là không có máy_sấy tóc trong phòng


{'FACILITIES#CLEANLINESS': 'None',
 'FACILITIES#MISCELLANEOUS': 'None',
 'FOOD&DRINKS#PRICES': 'None',
 'HOTEL#COMFORT': 'Positive',
 'HOTEL#PRICES': 'None',
 'ROOMS#COMFORT': 'None',
 'ROOM_AMENITIES#CLEANLINESS': 'None',
 'ROOM_AMENITIES#GENERAL': 'None',
 'SERVICE#GENERAL': 'Positive'}

# The End